In [1]:

import numpy as np

import pandas as pd


import matplotlib.pyplot as plt
import itertools as it

import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot


from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
init_notebook_mode(connected=True)

import os


## Border isolation

In this notebook we aim to extract the positions and intensity of border margins captured in images. Raw data consits of photographs of neuronal axons under a microscope (see README). Short transects were extracted using imageJ. Transects are vectors of intensity values and all vectors are of equal length. We seek to identify in each transect the position of cell membranes. 

We assume that the background is generally devoid of artefacs, so that membrane positions should correspond to first and last peaks in intensity.

Method:
We will rely on the mean shift clustering algorithm to extract peaks of intensity that correspond to border positions. 


**Index**

- **I. Extraction**
- **II. Visual inspection - a single transect**
- **III. Deployment across images and transects.**
- **IV. Amplitude Analysis** 



### I. Extraction.

- Image transect data was stored in the directory `data` in this same repository. 

the function `get_spectros` parses directories containing ImageJ transect data. It assumes that all directories in `data_dir` are of this kind. 

Image data is returned in the form of a dictionary with one entry per figure. Transect data per figure is captured in array format. 


In [2]:
from music_tools.mbrane_tools import (
    read_image, get_spectros
)

data_dir= 'data/'

image_dict= get_spectros(data_dir)

print('image Folders:')
image_dict.keys()


image Folders:


dict_keys(['1', '2', '3', '4', '5'])

### II. Visual inspecction.


We're going to select a single transect from one of the figure folders available to use. We will look at the distribution of intensities across this transect. 

We deploy on this transect the method that is to be applied across data sets. This is implemented through the function `frame_peaks`. This function takes a number of arguments that will impact the output:

- amp_cap: caps transect data to specified value.
- peak_cap: minimum intensity value accepted for cluster centroid positions. 
- peak_iso: minimum distance between cluster centroids: centroids below this distance are merged. 
- extremes: retains only positions corresponding to minimum and max peak positions. 


In [3]:
from music_tools.mbrane_tools import frame_obj, peak_finder


In [13]:

image_avail= image_dict.keys()
image_avail= sorted(image_avail)


## chose image folder by index
image_idx= 3
## chose transect by index (transects in increasing order).
frame= 10

peak_cap= .35 # min amp cap.

frame_plot= True
extremes= "extremes" ## extract only min and max peaks.
center_surface= True ## center peak positions. 
pxl_dist= 40e-5 # distance between pixels in transect, chose your metric

#################
#################

image_select= image_avail[image_idx]
print(image_select)


surface= image_dict[image_select]['surface'] # transect position list for selected image.

array_spec= image_dict[image_select]['I'].T

##################

pfinder= peak_finder(
    pxl_dist= pxl_dist,
    peak_cap= peak_cap
)

##################

pfinder.peaks(array_spec)

pfinder.extremes()
pfinder.centre()
pfinder.package()

pfinder.plot(idx= 7)



4


**Transect Figure**

### III. Deployment.

We deploy the method across transects and Figures.

We plot peak positions extracted by Figure. 

In [5]:
##########
from music_tools.my_music_tools import filter_output
 

pxl_dist= 40e-5

folder_out= "columns_stats/"
peak_cap= .35 # min amp cap.

extremes= "extremes"
center= True
standardize_amp= True

samp_track_dict= {}

pfinder= peak_finder(
    pxl_dist= pxl_dist,
    peak_cap= peak_cap
)

for image_idx in range(len(image_avail)):

    ts_list= []
    peaks= []
    amps= []

    #################
    #################

    image_select= image_avail[image_idx]
    print('folder: {}'.format(image_select))

    array_spec= image_dict[image_select]['I'].T
    surface= image_dict[image_select]['surface']
    
    ####### peak finder
    pfinder.peaks(array_spec)
    print(array_spec.shape)

    pfinder.extremes()
    pfinder.centre()
    pfinder.package()
    pfinder.write(filename= image_select + '_summ.txt', outdir= folder_out)
    samps_tracks= pfinder.get()
    
    pfinder.reset()
    ### plot
    
    output_peak_filter= filter_output(samps_tracks)
    
    samp_track_dict[image_select]= samps_tracks
    
    iplot(output_peak_filter)



folder: 1
(520, 36)


folder: 2
(375, 28)


folder: 3
(352, 32)


folder: 4
(352, 31)


folder: 5
(372, 28)


**Figure Peaks**

Centered peak positions across transects for data in each folder. Grandient shows intensity at peak position. Intensities were divided by the maximum by Figure. 

### IV. Amplitude analysis. 


We extracted the amplitude of transect peaks, divided by the maximum by folder. 

i) Peak amplitude by transect and folder: 

In [6]:
fig_amp= [go.Scatter(
    x= samp_track_dict[folder][:,0],
    y= samp_track_dict[folder][:,2],
    name= folder,
    mode= 'markers'
) for folder in samp_track_dict.keys()]

layout= go.Layout(
    title= 'slice intensities',
    xaxis= dict(
        title= 'xpos'    
    ),
    yaxis= dict(
        title= 'Intensity'
    )
)

Figure= go.Figure(data= fig_amp,layout= layout)
iplot(Figure)

**Figure - Intensity by transect and image folder**

ii) Averaged by positition.

We are going to assume that the initial and final transects in each image consist of the same same coordinates across biologica samples. We then standardize transect coordinates by picture to range between 0 and 1. 

To merge data from different pictures, because transect coordinates might not match, we use separate data into bins of the same size. Define the number of bins below. 

We plot amplitude average and standard deviation against axon length.

In [7]:

amps_means= [samp_track_dict[fold][:,[0,2]] for fold in samp_track_dict.keys()]
amps_means= tuple(amps_means)
amps_means= np.concatenate(amps_means, axis= 0)
amps_sizes= amps_means[:,0]


amps_dict= {
    z: [x for x in range(amps_means.shape[0]) if amps_sizes[x] == z] for z in list(set(amps_sizes))
}

amps_dict= {
    z: [amps_means[x,1] for x in g] for z,g in amps_dict.items()
}

amps_stats= {
    z: {
        'mean': np.median(g),
        'std': np.std(g)
    } for z,g in amps_dict.items()
}

############
############

surface= samp_track_dict[image_avail[0]][:,0]
sorted_bins= sorted(amps_stats.keys())

meanamps= [amps_stats[x]['mean'] for x in sorted_bins]
stdamp= [amps_stats[x]['std'] for x in sorted_bins]

fig_amp= [go.Scatter(
    x= sorted_bins,
    y= meanamps,
    error_y= dict(
        array= stdamp,
        type= 'data',
        #symmetric= True,
        visible=True
    ),
    mode= 'markers'
)]

layout= go.Layout(
    title= 'slice intensities',
    xaxis= dict(
        title= 'xpos'    
    ),
    yaxis= dict(
        title= 'Intensity'
    )
)

Figure= go.Figure(data= fig_amp,layout= layout)
iplot(Figure)

**Figure - Pooled Membrane intensities**